## 自動で抽出するアレ
- reference : [【https化に向けて】混在コンテンツのhttpを自動で抽出するやつを作った](http://www.procrasist.com/entry/get-http)

In [1]:
#coding: utf-8
from bs4 import BeautifulSoup
import urllib
from urllib import request
import time
import csv
import os
from argparse import ArgumentParser

def extract_urls(root_url):
    """
    トップページを指定すると、ブログ内に存在するurlをすべて抜き出してくれる
    """
    is_articles = True
    page = 1
    urls = []
    titles = []
    while is_articles:
        try:
            html = request.urlopen("{}/archive?page={}".format(root_url, page))
        except urllib.error.HTTPError as e: 
            # HTTPレスポンスのステータスコードが404, 403, 401などの例外処理
            print(e.reason)
            break
        except urllib.error.URLError as e: 
            # アクセスしようとしたurlが無効なときの例外処理
            print(e.reason)
            break
        soup = BeautifulSoup(html, "html.parser")
        articles = soup.find_all("a",class_="entry-title-link")
        for article in articles:
            urls.append(article.get("href"))
            titles.append(article.text)
        if len(articles) == 0:
            is_articles = False
        page += 1
    return urls, titles

def get_http(i, urls, titles):
    """
    各記事内のhttpを保存
    - gif, jpg, jpeg, png
    """
    url = urls[i]
    title = titles[i]
    print(url,title)
    try:
        html = request.urlopen(url)
    except urllib.error.HTTPError as e: 
        print(e.reason)
    except urllib.error.URLError as e: 
        print(e.reason)
    soup = BeautifulSoup(html, "html.parser")
    entry = soup.select(".entry-content")[0]
    imgs = entry.find_all("img")
    scripts = entry.find_all("script")
    for script in scripts:
        url = script.get("src")
        if url == None:
            continue
        if "http://" in url:
                print("\tscript tag:",url)
    for img in imgs:
        url = img.get("src")
        if "http://" in url:
            print("\timg tag:",url)

def get_http_all(urls, titles):
    """
    各記事内のhttpを保存
    - gif, jpg, jpeg, png
    """
    for i, (url,title) in enumerate(zip(urls, titles)):
        count = 0
        try:
            html = request.urlopen(url)
        except urllib.error.HTTPError as e: 
            print(e.reason)
        except urllib.error.URLError as e: 
            print(e.reason)
        soup = BeautifulSoup(html, "html.parser")
        entry = soup.select(".entry-content")[0]
        imgs = entry.find_all("img")
        scripts = entry.find_all("script")
        for script in scripts:
            url = script.get("src")
            if url == None:
                continue
            if "http://" in url:
                if count == 0:
                    print(title,url)
                print("\tscript tag:",url)
                count += 1
        for img in imgs:
            url = img.get("src")
            if "http://" in url:
                if count == 0:
                    print(title,url)
                print("\timg tag:",url)
                count += 1

  ## この下にある、urlに自分のアドレスを入力して実行(必須)
 

In [2]:
url = "http://www.procrasist.com"
urls, titles = extract_urls(url)
for i, (url,title) in enumerate(zip(urls, titles)):
    print(i, title, url)

0 自サイトの『関連記事』を自動で探すやつを作った(バスケット分析) http://www.procrasist.com/entry/extract-related-articles
1 ipywidgetsとbokehで『jupyter』の更なる高みへ 【インタラクティブなグラフ描画】 http://www.procrasist.com/entry/jupyter-more-interactive
2 【主観】海外旅行をする前に読んでおくべき人気の漫画・アニメ集 http://www.procrasist.com/entry/foreign-manga
3 海外旅行で超絶助かったおすすめアプリ10選！ http://www.procrasist.com/entry/travel-app
4 jupyterのテーマカラーが超簡単にかっこよくなる『jupyterthemes』 http://www.procrasist.com/entry/2017/09/18/130300
5 【Amazon】Kindle Unlimitedのオススメの本と利用法を模索してまとめた！ http://www.procrasist.com/entry/kindle-unlimited
6 【ビットコイン】仮想通貨の全銘柄を集められるAPIを使って分析してみた http://www.procrasist.com/entry/cryptcoin-api_
7 ぼくのなつやすみ http://www.procrasist.com/entry/summer-vacation
8 数字に騙されるな！リテラシーを身につけるための事例5選 http://www.procrasist.com/entry/science-literacy
9 【NoTube】YouTubeとTwitter廃人を矯正するchrome拡張を作ったよ http://www.procrasist.com/entry/notube
10 【ホームラン連発！】甲子園第99回大会のホームラン記録をまとめたよ http://www.procrasist.com/entry/homerun-2017
11 【図解】なぜみんなの意見を取り入れた資料はダメになるのか http://www.procrasist.com/entry/every-

## 全記事のhttpを取得したい方はこちらを実行

In [3]:
get_http_all(urls, titles)

【図解】なぜみんなの意見を取り入れた資料はダメになるのか http://chart.apis.google.com/chart?cht=tx&chl=%5Ctheta
	img tag: http://chart.apis.google.com/chart?cht=tx&chl=%5Ctheta
コピペで(ほぼ)一発！jQueryでオシャレな地図ツールを作ったよ。 http://code.jquery.com/jquery-1.11.3.min.js
	script tag: http://code.jquery.com/jquery-1.11.3.min.js
	script tag: http://jvectormap.com/js/jquery-jvectormap-2.0.3.min.js
	script tag: http://jvectormap.com/js/jquery-jvectormap-world-merc.js
jQueryでブログ内にオシャレな地図を表示したいんじゃ http://ajax.googleapis.com/ajax/libs/jquery/1.10.2/jquery.min.js
	script tag: http://ajax.googleapis.com/ajax/libs/jquery/1.10.2/jquery.min.js
	script tag: http://code.highcharts.com/highcharts.js
	script tag: http://code.jquery.com/jquery-1.11.3.min.js
	script tag: http://takemaru-hirai.github.io/japan-map/example/js/jquery.min.js
	script tag: http://takemaru-hirai.github.io/japan-map/example/js/jquery.japan-map.min.js
もう試験で困らない！√2の求め方10選 http://chart.apis.google.com/chart?cht=tx&chl=%5Csqrt%7B2%7D
	img tag: http://chart.apis.google.com/cha

ファミレスのアレの体積を求めてみた http://chart.apis.google.com/chart?cht=tx&chl=x%20%3D%20t
	img tag: http://chart.apis.google.com/chart?cht=tx&chl=x%20%3D%20t
	img tag: http://chart.apis.google.com/chart?cht=tx&chl=%5Cfrac%7Ba%2Bb%7D%7B2%7D%5Cpi%20r%5E%7B2%7D
	img tag: http://chart.apis.google.com/chart?cht=tx&chl=%5Cpi%20r%5E%7B2%7D%20%28a%2Bb%29%2F2%20%3D%20%5Cfrac%7Ba%2Bb%7D%7B2%7D%20%5Cpi%20r%5E%7B2%7D
	img tag: http://chart.apis.google.com/chart?cht=tx&chl=%20%5Cfrac%7B5.5%2B8.5%7D%7B2%7D%5Ctimes%5Cpi%20%5Ctimes3.0%5E%7B2%7D%3D197.82
	img tag: http://chart.apis.google.com/chart?cht=tx&chl=%5Cpi%3D3.14
機械学習で使う指標総まとめ(教師あり学習編) http://chart.apis.google.com/chart?cht=tx&chl=%7BAccuracy%20%3D%20%5Cfrac%7BTP%2BTN%7D%7BTP%2BFP%2BTN%2BFN%7D%7D
	img tag: http://chart.apis.google.com/chart?cht=tx&chl=%7BAccuracy%20%3D%20%5Cfrac%7BTP%2BTN%7D%7BTP%2BFP%2BTN%2BFN%7D%7D
	img tag: http://chart.apis.google.com/chart?cht=tx&chl=Precision%3D%5Cfrac%7BTP%7D%7BTP%2BFP%7D
	img tag: http://chart.apis.google.com/chart?

無機質な色に和を。『NIPPON COLORS』という色見本サイトが素敵。 http://cdn.image.st-hatena.com/image/scale/2079337f1dacb9b5b8a13193631d436bf3fcf7f7/enlarge=0;height=200;version=1;width=200/http%3A%2F%2Fis3.mzstatic.com%2Fimage%2Fthumb%2FPurple%2Fv4%2Ff2%2Fa9%2Fe6%2Ff2a9e644-660f-e2d5-891b-b17f599800f5%2Fsource%2F100x100bb.jpg
	img tag: http://cdn.image.st-hatena.com/image/scale/2079337f1dacb9b5b8a13193631d436bf3fcf7f7/enlarge=0;height=200;version=1;width=200/http%3A%2F%2Fis3.mzstatic.com%2Fimage%2Fthumb%2FPurple%2Fv4%2Ff2%2Fa9%2Fe6%2Ff2a9e644-660f-e2d5-891b-b17f599800f5%2Fsource%2F100x100bb.jpg
【6年ぶり306回目】素数年2017年は◯◯な年！！まとめ http://chart.apis.google.com/chart?cht=tx&chl=%7B%0Aa%5E%7B2%7D%2Bb%5E%7B2%7D%3Dc%5E%7B2%7D%20%28a%2Cb%2Cc%E3%81%AF%E8%87%AA%E7%84%B6%E6%95%B0%29%7D
	img tag: http://chart.apis.google.com/chart?cht=tx&chl=%7B%0Aa%5E%7B2%7D%2Bb%5E%7B2%7D%3Dc%5E%7B2%7D%20%28a%2Cb%2Cc%E3%81%AF%E8%87%AA%E7%84%B6%E6%95%B0%29%7D
	img tag: http://chart.apis.google.com/chart?cht=tx&chl=%7B%0A2017%5E%7B2%7D%3D792%5E%

研究・プログラミングメモはもうこれで完璧。markdownもtexもpythonを一緒に使える jupyterが超便利!! http://chart.apis.google.com/chart?cht=tx&chl=%7B%20%5Cdisplaystyle%0Ay%3D%5Csin%7B%5Calpha%20x%7D%7D
	img tag: http://chart.apis.google.com/chart?cht=tx&chl=%7B%20%5Cdisplaystyle%0Ay%3D%5Csin%7B%5Calpha%20x%7D%7D
はてなブログでMarkdownを使うときのTips集 http://chart.apis.google.com/chart?cht=tx&chl=%7B%20%5Cdisplaystyle%0Ab_n%20%3D%20%5Csum_%7Bm%3D0%7D%5E%7BN-1%7D%20a_m%0A%7D
	img tag: http://chart.apis.google.com/chart?cht=tx&chl=%7B%20%5Cdisplaystyle%0Ab_n%20%3D%20%5Csum_%7Bm%3D0%7D%5E%7BN-1%7D%20a_m%0A%7D


## ひと記事ずつ見たい方はこちらを実行
- 先ほどのリストに振られた番号を見ましょう
- i に見たい番号を代入しましょう

In [5]:
i = 10
get_http(i, urls, titles)

http://www.procrasist.com/entry/homerun-2017 【ホームラン連発！】甲子園第99回大会のホームラン記録をまとめたよ
